In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from shapely.geometry import Point
from scipy.spatial.distance import cdist
import networkx as nx

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-state-boundaries/cb_2018_us_state_5m/cb_2018_us_state_5m.shp
/kaggle/input/us-state-boundaries/cb_2018_us_state_5m/cb_2018_us_state_5m.cpg
/kaggle/input/us-state-boundaries/cb_2018_us_state_5m/cb_2018_us_state_5m.prj
/kaggle/input/us-state-boundaries/cb_2018_us_state_5m/cb_2018_us_state_5m.shx
/kaggle/input/us-state-boundaries/cb_2018_us_state_5m/cb_2018_us_state_5m.shp.iso.xml
/kaggle/input/us-state-boundaries/cb_2018_us_state_5m/cb_2018_us_state_5m.dbf
/kaggle/input/us-state-boundaries/cb_2018_us_state_5m/cb_2018_us_state_5m.shp.ea.iso.xml
/kaggle/input/all-oncologists-lat-long/all_oncologists_lat_long.csv
/kaggle/input/us-doctors-and-clinicians-2021/tl_2019_us_zcta510.shp
/kaggle/input/us-doctors-and-clinicians-2021/DAC_NationalDownloadableFile.csv
/kaggle/input/us-doctors-and-clinicians-2021/tl_2019_us_zcta510.shx
/kaggle/input/us-doctors-and-clinicians-2021/physicians-by-specialty-and-patient-care-hours.csv
/kaggle/input/us-doctors-and-clinicians-2021/tl_2019_us_z

In [2]:
# Read 'train.csv' dataset and preview
raw_df = pd.read_csv('/kaggle/input/us-doctors-and-clinicians-2021/DAC_NationalDownloadableFile.csv', encoding= 'unicode_escape', low_memory=False)
raw_df.head()

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,adr_ln_1,adr_ln_2,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id
0,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,NaN,NaN,M,NaN,OTHER,...,901 HARRY S TRUMAN DR,NaN,NaN,LARGO,MD,207745477,2.406771e+09,Y,Y,MD207745477LA901XXDRXX500
1,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,NaN,NaN,M,NaN,OTHER,...,1850 TOWN CTR PKWY,NaN,NaN,RESTON,VA,201903204,2.406862e+09,Y,Y,VA201903204RE1850XPKWY400
2,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,NaN,NaN,M,NaN,OTHER,...,1701 N GEORGE MASON DR,NaN,NaN,ARLINGTON,VA,222053610,7.035586e+09,Y,Y,VA222053610AR1701XDRXX500
3,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,NaN,NaN,M,NaN,OTHER,...,14207 PARK CTR DR,SUITE 102,NaN,LAUREL,MD,207075255,3.016045e+09,Y,Y,MD207075255LA14207DRXX402
4,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,NaN,NaN,M,NaN,OTHER,...,8600 OLD GEORGETOWN RD,NaN,NaN,BETHESDA,MD,208141422,NaN,Y,Y,MD208141422BE8600XRDXX400


In [3]:
raw_df.dtypes

NPI               int64
Ind_PAC_ID        int64
Ind_enrl_ID      object
lst_nm           object
frst_nm          object
mid_nm           object
suff             object
gndr             object
Cred             object
Med_sch          object
Grd_yr          float64
pri_spec         object
sec_spec_1       object
sec_spec_2       object
sec_spec_3       object
sec_spec_4       object
sec_spec_all     object
Telehlth         object
org_nm           object
org_pac_id      float64
num_org_mem     float64
adr_ln_1         object
adr_ln_2         object
ln_2_sprs        object
cty              object
st               object
zip              object
phn_numbr       float64
ind_assgn        object
grp_assgn        object
adrs_id          object
dtype: object

[Data dictionary for Doctors and Clinicians dataset](https://data.cms.gov/provider-data/sites/default/files/data_dictionaries/physician/DOC_Data_Dictionary.pdf)

The difference between a doctor and a clinician is in their level of education. Clinicians are all healthcare providers who deal directly with patients, while doctors are clinicians who focus on a particular specialty involving non-surgical treatment.

There are up to 3 identifiers for a given doctor [(*more details here*)](https://blog.evalyticslabs.com/differences-between-npi-pac-id-and-enrollment-id-982a0d0d738e) 
1. **NPI:** The National Provider Identifier (NPI) is a 10-digit identification number that is assigned to healthcare providers by the National Plan and Provider Enumeration System (NPPES).
2. **PAC ID:** The Provider Advertising ID (PAC ID) is a unique identifier assigned to healthcare providers by the Centers for Medicare and Medicaid Services (CMS). It is used to identify the provider during the advertisements to patients.
3. **Enrollment ID:** Enrollment ID is a unique identifier assigned to healthcare providers by Medicare and Medicaid during the enrollment process for these government healthcare programs. It is used to identify providers when submitting claims for reimbursement or checking on the status of a claim.

## Exploratory Data Analysis

Check unique count for NPI, PAC ID, and Enrollment ID

In [4]:
# check unique count for NPI, PAC ID, and Enrollment ID
raw_df['NPI'].value_counts()

1205003480    583
1346219185    136
1326534264    135
1508099425     94
1770512105     93
             ... 
1467684894      1
1467684902      1
1467685107      1
1467685149      1
1992999874      1
Name: NPI, Length: 1257302, dtype: int64

In [5]:
# check unique count for PAC ID
raw_df['Ind_PAC_ID'].value_counts()

4688845282    583
7517032493    136
9133476245    135
3577601020     94
5496762569     93
             ... 
345475612       1
6002952918      1
2961782214      1
4486794245      1
8820168917      1
Name: Ind_PAC_ID, Length: 1257307, dtype: int64

In [6]:
# check unique count for Enrollment ID
raw_df['Ind_enrl_ID'].value_counts()

I20110912000134    583
I20080813000119    134
I20180720000691    133
I20091103000772     94
I20200908002933     93
                  ... 
I20160902002072      1
I20170220001923      1
I20161013000831      1
I20191106000260      1
I20111027000796      1
Name: Ind_enrl_ID, Length: 1319664, dtype: int64

From the results above, it can be seen that some doctors/clinicians practice in multiple locations (one of them has 583 locations). Let's take a look at that.

In [7]:
# look into clinician with NPI value 1205003480
most_practices_doctor_df = raw_df.loc[raw_df['NPI'] == 1205003480]
most_practices_doctor_df.head()

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,adr_ln_1,adr_ln_2,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id
495869,1205003480,4688845282,I20110912000134,XENOS,HELEN,NaN,NaN,F,NaN,OTHER,...,181 W MADISON ST,SUITE 3825,NaN,CHICAGO,IL,606024500,8.886636e+09,Y,Y,IL606024500CH181XXSTXX402
495870,1205003480,4688845282,I20110912000134,XENOS,HELEN,NaN,NaN,F,NaN,OTHER,...,415 N LA SALLE DR,SUITE 100,NaN,CHICAGO,IL,606542730,3.122192e+09,Y,Y,IL606542730CH415XXDRXX501
495871,1205003480,4688845282,I20110912000134,XENOS,HELEN,NaN,NaN,F,NaN,OTHER,...,6300 N RIDGE AVE,NaN,NaN,CHICAGO,ID,606601099,7.739736e+09,Y,Y,ID606601099CH6300XAVEX400
495872,1205003480,4688845282,I20110912000134,XENOS,HELEN,NaN,NaN,F,NaN,OTHER,...,2501 N CHESTNUT AV,NaN,NaN,ARLINGTON HEIGHTS,IL,600042401,8.473942e+09,Y,Y,IL600042401AR2501XAVXX400
495873,1205003480,4688845282,I20110912000134,XENOS,HELEN,NaN,NaN,F,NaN,OTHER,...,800 W OAKTON ST,NaN,NaN,ARLINGTON HEIGHTS,IL,600044602,8.472534e+09,Y,Y,IL600044602AR800XXSTXX400


In [8]:
# confirm that the last name and first name are the only values
print(most_practices_doctor_df['lst_nm'].value_counts())
print(most_practices_doctor_df['frst_nm'].value_counts())

XENOS    583
Name: lst_nm, dtype: int64
HELEN    583
Name: frst_nm, dtype: int64


Since this project is focused on tumor doctor recommendations, drop all clinicians that do not have a credential or do not have a credential for treating tumors.

In [9]:
# check unique count for credentials
raw_df['Cred'].value_counts()

MD     417582
PA      39858
NP      25509
DO      25298
CNA     18671
DC      13166
PT      12330
OD      12291
DPM      8387
CSW      7674
CP       6231
OT       1311
AU       1176
CNS       981
CNM       822
MNT       814
AA        427
DDS       417
DDM       209
PSY       112
SCW         2
Name: Cred, dtype: int64

* MD - Medical Doctor
* PA - Physician Assistant
* NP - Nurse Practitioner
* DO - Osteopathic Doctor
* CNA - Certified Nursing Assistant
* DC - Chiropractic Doctor
* PT - Physical Therapist
* OD - Optometrist
* DPM - Doctor of Podiatric Medicine (Foot)
* CSW - Clinical Social Worker
* CP - Care Partner
* OT - Occupational Therapist
* AU - Doctor of Audiology
* CNS - Clinical Nurse Specialist
* CNM - Certified Nurse-Midwife
* MNT - Medical Nutrition Therapist
* AA - Anesthesiologist Assistant
* DDS - Doctor of Dental Surgery
* DDM - Doctor of Dental Medicine
* PSY - Doctor of Psychology
* SCW - Social Worker?

In [10]:
# create a full name column formatted by last name, first name middle initial
# raw_df['full_name'] = raw_df['lst_nm'] + ',' + raw_df['frst_nm'] + ' ' + raw_df['mid_nm']

# fill the mid_nm column with blanks instead of NaNs
raw_df['mid_nm'] = raw_df['mid_nm'].fillna('')

raw_df['full_name'] = raw_df['lst_nm'] + ', ' + raw_df['frst_nm'] + ' ' + raw_df['mid_nm'].replace('', '')

raw_df['full_name'].value_counts()

XENOS, HELEN               583
CLINE, LAURA               138
BOX, DAVID F               136
KUHN, YVONNE V              94
BROCKMAN, MICHAEL           93
                          ... 
GOLDBERG, RYAN J             1
HOOGESTEGER, AMANDA S        1
ZABLOTNEY, JASON EDWARD      1
RUDASILL, JENNIFER C         1
JOFFE, GABRIELLA T           1
Name: full_name, Length: 1217590, dtype: int64

In [11]:
# only get the first five digits of the zip code
raw_df['zip'] = raw_df['zip'].str[:5]

# fill in leading zeros for zip codes with 4 digits
raw_df['zip'] = raw_df['zip'].apply(lambda x: x.zfill(5))

# convert back to int
raw_df['zip'] = raw_df['zip'].astype(int)

raw_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,adr_ln_2,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name
0,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,NaN,LARGO,MD,20774,2.406771e+09,Y,Y,MD207745477LA901XXDRXX500,"ENKESHAFI, ARDALAN"
1,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,NaN,RESTON,VA,20190,2.406862e+09,Y,Y,VA201903204RE1850XPKWY400,"ENKESHAFI, ARDALAN"
2,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,NaN,ARLINGTON,VA,22205,7.035586e+09,Y,Y,VA222053610AR1701XDRXX500,"ENKESHAFI, ARDALAN"
3,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,SUITE 102,NaN,LAUREL,MD,20707,3.016045e+09,Y,Y,MD207075255LA14207DRXX402,"ENKESHAFI, ARDALAN"
4,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,NaN,BETHESDA,MD,20814,NaN,Y,Y,MD208141422BE8600XRDXX400,"ENKESHAFI, ARDALAN"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481939,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,NaN,NaN,FEDERAL WAY,WA,98003,2.538382e+09,Y,Y,WA980036208FE33501SXXX400,"DESCHENES, GEOFFREY R"
2481940,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,NaN,NaN,SEATTLE,WA,98101,2.062237e+09,Y,Y,WA981012742SE925XXSTXX300,"DESCHENES, GEOFFREY R"
2481941,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,NaN,NaN,SEATTLE,WA,98101,NaN,Y,Y,WA981012756SE1100XAVEX300,"DESCHENES, GEOFFREY R"
2481942,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,SUITE 108,NaN,SEATTLE,WA,98105,2.065258e+09,Y,Y,WA981053999SE4575XNEXX502,"DESCHENES, GEOFFREY R"


In [12]:
# fill the adr_ln_2 column with blanks instead of NaNs
raw_df['adr_ln_2'] = raw_df['adr_ln_2'].fillna('')

# create a full address column
raw_df['full_address'] = raw_df['adr_ln_1'] + ' ' + raw_df['adr_ln_2'] + ', ' + raw_df['cty'] + ', ' + raw_df['st'] + ' ' + raw_df['zip'].astype(str)

# remove any spaces before the comma
raw_df['full_address'] = raw_df['full_address'].str.replace(' ,', ',')

raw_df['full_address']

0                     901 HARRY S TRUMAN DR, LARGO, MD 20774
1                       1850 TOWN CTR PKWY, RESTON, VA 20190
2                1701 N GEORGE MASON DR, ARLINGTON, VA 22205
3              14207 PARK CTR DR SUITE 102, LAUREL, MD 20707
4                 8600 OLD GEORGETOWN RD, BETHESDA, MD 20814
                                 ...                        
2481939               33501 1ST WAY S, FEDERAL WAY, WA 98003
2481940                     925 SENECA ST, SEATTLE, WA 98101
2481941                      1100 9TH AVE, SEATTLE, WA 98101
2481942    4575 SAND POINT WAY NE SUITE 108, SEATTLE, WA ...
2481943                 1602 SKIPWITH RD, RICHMOND, VA 23229
Name: full_address, Length: 2481944, dtype: object

In [13]:
# convert phone numbers in scientific notation to 10 digit numbers and replace missing phone numbers with 'missing'
raw_df['phn_numbr'] = raw_df['phn_numbr'].fillna(0).astype(int).replace(0, 'missing')
raw_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
0,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,LARGO,MD,20774,2406771000,Y,Y,MD207745477LA901XXDRXX500,"ENKESHAFI, ARDALAN","901 HARRY S TRUMAN DR, LARGO, MD 20774"
1,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,RESTON,VA,20190,2406862300,Y,Y,VA201903204RE1850XPKWY400,"ENKESHAFI, ARDALAN","1850 TOWN CTR PKWY, RESTON, VA 20190"
2,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,ARLINGTON,VA,22205,7035586161,Y,Y,VA222053610AR1701XDRXX500,"ENKESHAFI, ARDALAN","1701 N GEORGE MASON DR, ARLINGTON, VA 22205"
3,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,LAUREL,MD,20707,3016045254,Y,Y,MD207075255LA14207DRXX402,"ENKESHAFI, ARDALAN","14207 PARK CTR DR SUITE 102, LAUREL, MD 20707"
4,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,BETHESDA,MD,20814,missing,Y,Y,MD208141422BE8600XRDXX400,"ENKESHAFI, ARDALAN","8600 OLD GEORGETOWN RD, BETHESDA, MD 20814"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481939,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,NaN,FEDERAL WAY,WA,98003,2538382400,Y,Y,WA980036208FE33501SXXX400,"DESCHENES, GEOFFREY R","33501 1ST WAY S, FEDERAL WAY, WA 98003"
2481940,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,NaN,SEATTLE,WA,98101,2062236600,Y,Y,WA981012742SE925XXSTXX300,"DESCHENES, GEOFFREY R","925 SENECA ST, SEATTLE, WA 98101"
2481941,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,NaN,SEATTLE,WA,98101,missing,Y,Y,WA981012756SE1100XAVEX300,"DESCHENES, GEOFFREY R","1100 9TH AVE, SEATTLE, WA 98101"
2481942,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,NaN,SEATTLE,WA,98105,2065258000,Y,Y,WA981053999SE4575XNEXX502,"DESCHENES, GEOFFREY R","4575 SAND POINT WAY NE SUITE 108, SEATTLE, WA ..."


In [14]:
unique_clinicians_df = raw_df.drop_duplicates(subset='full_name', keep='first')
unique_clinicians_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
0,1003000126,7517003643,I20130530000085,ENKESHAFI,ARDALAN,,NaN,M,NaN,OTHER,...,NaN,LARGO,MD,20774,2406771000,Y,Y,MD207745477LA901XXDRXX500,"ENKESHAFI, ARDALAN","901 HARRY S TRUMAN DR, LARGO, MD 20774"
7,1003000134,4284706367,I20080707000385,CIBULL,THOMAS,L,NaN,M,NaN,UNIVERSITY OF KENTUCKY COLLEGE OF MEDICINE,...,NaN,GLENVIEW,IL,60026,missing,Y,Y,IL600261301GL2100XRDXX300,"CIBULL, THOMAS L","2100 PFINGSTEN RD, GLENVIEW, IL 60026"
14,1003000142,9931380672,I20110801000609,KHALIL,RASHID,,NaN,M,NaN,OTHER,...,NaN,TOLEDO,OH,43608,missing,Y,Y,OH436082603TO2213XSTXX300,"KHALIL, RASHID","2213 CHERRY ST, TOLEDO, OH 43608"
20,1003000423,9133397268,I20110715000515,VELOTTA,JENNIFER,A,NaN,F,NaN,TOLEDO MEDICAL COLLEGE,...,NaN,MENTOR,OH,44060,4403577100,Y,Y,OH440608702ME9500XAVEX301,"VELOTTA, JENNIFER A","9500 MENTOR AVE SUITE 220, MENTOR, OH 44060"
21,1003000480,446348254,I20071113000017,ROTHCHILD,KEVIN,B,NaN,M,NaN,OHIO STATE UNIVERSITY COLLEGE OF MEDICINE,...,NaN,AURORA,CO,80045,7208480000,Y,Y,CO800452541AU1635XCTXX300,"ROTHCHILD, KEVIN B","1635 AURORA CT, AURORA, CO 80045"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481935,1992999569,4385709583,I20120120000727,YUEN,JENNY,FEI C,NaN,F,NaN,UNIVERSITY OF CALIFORNIA - SCHOOL OF OPTOMETRY,...,NaN,SAN FRANCISCO,CA,94102,4156032020,Y,M,CA941024417SA515XXSTXX300,"YUEN, JENNY FEI C","515 GOUGH ST, SAN FRANCISCO, CA 94102"
2481936,1992999791,1254502982,I20110928000849,THOMAS,SHELIA,KING,NaN,F,NaN,OTHER,...,NaN,MEMPHIS,TN,38119,9016829595,Y,Y,TN381194713ME6266XAVEX300,"THOMAS, SHELIA KING","6266 POPLAR AVE, MEMPHIS, TN 38119"
2481937,1992999817,7113191032,I20111110000630,TAKENISHI,GREG,S,NaN,M,NaN,MEDICAL COLLEGE OF WISCONSIN,...,NaN,STOCKTON,CA,95210,2094762000,Y,Y,CA952103377ST7373XLANE300,"TAKENISHI, GREG S","7373 W LANE, STOCKTON, CA 95210"
2481939,1992999825,143414284,I20101102000933,DESCHENES,GEOFFREY,R,NaN,M,NaN,JEFFERSON MEDICAL COLLEGE OF THOMAS JEFFERSON ...,...,NaN,FEDERAL WAY,WA,98003,2538382400,Y,Y,WA980036208FE33501SXXX400,"DESCHENES, GEOFFREY R","33501 1ST WAY S, FEDERAL WAY, WA 98003"


**There are 865,283 unique clinicians (some of which are MDs and DOs) in this dataset**

Only keep the medical doctors (MDs) and osteopathic doctors (DOs) in the data

In [15]:
doctors_df = raw_df.loc[(raw_df['Cred'] == 'MD') | (raw_df['Cred'] == 'DO')]
doctors_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
74,1003002312,547203192,I20051018001123,HOPKINS,PATRICIA,T,NaN,F,MD,OTHER,...,NaN,WEYMOUTH,MA,2190,6177739198,Y,Y,MA021901843WE571XXSTXX300,"HOPKINS, PATRICIA T","571 MAIN ST, WEYMOUTH, MA 2190"
111,1003003856,6305813148,I20040913000368,SANTIAGO,CYNTHIA,,NaN,F,MD,NEW YORK UNIVERSITY SCHOOL OF MEDICINE,...,NaN,PORT JEFFERSON STATION,NY,11776,6317326984,Y,M,NY117763336PO5XXXXAVEX300,"SANTIAGO, CYNTHIA","5 ROOSEVELT AVE, PORT JEFFERSON STATION, NY 11776"
112,1003003856,6305813148,I20040913000368,SANTIAGO,CYNTHIA,,NaN,F,MD,NEW YORK UNIVERSITY SCHOOL OF MEDICINE,...,NaN,SELDEN,NY,11784,6317326984,Y,M,NY117842569SE260XXRDXX401,"SANTIAGO, CYNTHIA","260 MIDDLE COUNTRY RD SUITE 6, SELDEN, NY 11784"
113,1003003856,6305813148,I20040913000368,SANTIAGO,CYNTHIA,,NaN,F,MD,NEW YORK UNIVERSITY SCHOOL OF MEDICINE,...,NaN,NEW CITY,NY,10956,2015652533,Y,Y,NY109565101NE3XXXXRDXX401,"SANTIAGO, CYNTHIA","3 E EVERGREEN RD SUITE 1017, NEW CITY, NY 10956"
119,1003004540,7416917000,I20041018000934,CABANELLAS,JENNINE,M,NaN,F,MD,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,...,NaN,BOYNTON BEACH,FL,33472,5617778946,M,M,FL334722900BO8198XRDXX403,"CABANELLAS, JENNINE M","8198 S JOG RD SUITE 102, BOYNTON BEACH, FL 33472"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481783,1992995765,446246276,I20040426000309,APPEL,DEBRA,A,NaN,F,MD,BAYLOR COLLEGE OF MEDICINE,...,NaN,VALHALLA,NY,10595,9149099018,Y,Y,NY105951530VA100XXRDXX303,"APPEL, DEBRA A","100 WOODS RD MACY PAVILLION, VALHALLA, NY 10595"
2481784,1992995765,446246276,I20040426000309,APPEL,DEBRA,A,NaN,F,MD,BAYLOR COLLEGE OF MEDICINE,...,NaN,VALHALLA,NY,10595,9144936940,Y,Y,NY105951652VA95XXXRDXX300,"APPEL, DEBRA A","95 GRASSLANDS RD, VALHALLA, NY 10595"
2481785,1992995765,446246276,I20040426000309,APPEL,DEBRA,A,NaN,F,MD,BAYLOR COLLEGE OF MEDICINE,...,NaN,MOUNT KISCO,NY,10549,9146661371,Y,Y,NY105493417MO400XXSTXX400,"APPEL, DEBRA A","400 E MAIN ST, MOUNT KISCO, NY 10549"
2481837,1992997332,5496776064,I20051209000751,ROTH,MARA,Y,NaN,F,MD,OTHER,...,NaN,SEATTLE,WA,98104,2067313000,Y,Y,WA981042420SE325XXAVEX300,"ROTH, MARA Y","325 9TH AVE, SEATTLE, WA 98104"


In [16]:
doctors_df['Cred'].value_counts()

MD    417582
DO     25298
Name: Cred, dtype: int64

In [17]:
# check if a doctor has a multiple IDs, meaning that he/she/they have multiple practices
# check if 'full_name' value has multiple values in NPI, PAC ID, or Enrollment ID columns
# group by the doctor name and the occurences in another column and then sum (True = 1)

doctors_npi = doctors_df.groupby('full_name')['NPI'].nunique() > 1
count_multiple_npi = doctors_npi.sum()
print('Doctors with multiple NPIs:', count_multiple_npi)

doctors_pacid = doctors_df.groupby('full_name')['Ind_PAC_ID'].nunique() > 1
count_multiple_pacid = doctors_pacid.sum()
print('Doctors with multiple PAC IDs:', count_multiple_pacid)

doctors_enrollmentid = doctors_df.groupby('full_name')['Ind_enrl_ID'].nunique() > 1
count_multiple_enrollmentid = doctors_enrollmentid.sum()
print('Doctors with multiple Enrollment IDs:', count_multiple_enrollmentid)

Doctors with multiple NPIs: 1798
Doctors with multiple PAC IDs: 1798
Doctors with multiple Enrollment IDs: 13209


In [18]:
# check NPI, display only first 100
doctors_multiple_npi = doctors_npi[doctors_npi].index.tolist()
print('Doctors with multiple NPIs:', doctors_multiple_npi[:100])

Doctors with multiple NPIs: ['ABBAS, GHULAM ', 'ABDULLAH, MUHAMMAD ', 'ABRAHAM, GEORGE ', 'ADLER, MICHAEL J', 'AGARWAL, ANURAG ', 'AGGARWAL, ARCHANA ', 'AGGARWAL, ATUL ', 'AGGARWAL, SANJAY K', 'AGRAWAL, VIVEK ', 'AGUILAR, DAVID ', 'AHMAD, ABRAR ', 'AHMAD, ARIF ', 'AHMAD, AZIZ ', 'AHMAD, IJAZ ', 'AHMAD, IMTIAZ ', 'AHMAD, MUNIR ', 'AHMAD, NAILA ', 'AHMAD, SALMAN ', 'AHMAD, SYED ', 'AHMAD, TANVEER ', 'AHMAD, TANVIR ', 'AHMAD, WASEEM ', 'AHMED, ANWAR ', 'AHMED, BASHIR ', 'AHMED, FAWAD ', 'AHMED, IMRAN ', 'AHMED, IMTIAZ ', 'AHMED, JAMIL ', 'AHMED, MANSOOR ', 'AHMED, MAQBOOL ', 'AHMED, MOHAMED A', 'AHMED, MOHAMMED ', 'AHMED, MUHAMMAD A', 'AHMED, NADEEM ', 'AHMED, NAEEM ', 'AHMED, SHAKEEL ', 'AHMED, SHAKIL ', 'AHMED, SYED A', 'AHMED, ZULFIQAR ', 'AHN, SAM S', 'AKBAR, MUHAMMAD S', 'AKHTAR, NAVEED ', 'ALAM, MUHAMMAD M', 'ALEXANDER, CHRISTOPHER ', 'ALI, AMJAD ', 'ALI, MAHMOOD ', 'ALI, MIR A', 'ALI, MIR M', 'ALI, MIR T', 'ALI, MOHAMED S', 'ALI, MOHAMMED ', 'ALI, MUHAMMAD ', 'ALI, SYED ', 'ALI, SY

In [19]:
# check for duplicate FULL names
npi_has_duplicate_full_names = len(doctors_multiple_npi) != len(set(doctors_multiple_npi))
print('Doctors with same FULL name:', npi_has_duplicate_full_names)

Doctors with same FULL name: False


In [20]:
# check PAC ID, display only first 100
doctors_multiple_pacid = doctors_pacid[doctors_pacid].index.tolist()
print('Doctors with multiple PAC IDs:', doctors_multiple_pacid[:100])

Doctors with multiple PAC IDs: ['ABBAS, GHULAM ', 'ABDULLAH, MUHAMMAD ', 'ABRAHAM, GEORGE ', 'ADLER, MICHAEL J', 'AGARWAL, ANURAG ', 'AGGARWAL, ARCHANA ', 'AGGARWAL, ATUL ', 'AGGARWAL, SANJAY K', 'AGRAWAL, VIVEK ', 'AGUILAR, DAVID ', 'AHMAD, ABRAR ', 'AHMAD, ARIF ', 'AHMAD, AZIZ ', 'AHMAD, IJAZ ', 'AHMAD, IMTIAZ ', 'AHMAD, MUNIR ', 'AHMAD, NAILA ', 'AHMAD, SALMAN ', 'AHMAD, SYED ', 'AHMAD, TANVEER ', 'AHMAD, TANVIR ', 'AHMAD, WASEEM ', 'AHMED, ANWAR ', 'AHMED, BASHIR ', 'AHMED, FAWAD ', 'AHMED, IMRAN ', 'AHMED, IMTIAZ ', 'AHMED, JAMIL ', 'AHMED, MANSOOR ', 'AHMED, MAQBOOL ', 'AHMED, MOHAMED A', 'AHMED, MOHAMMED ', 'AHMED, MUHAMMAD A', 'AHMED, NADEEM ', 'AHMED, NAEEM ', 'AHMED, SHAKEEL ', 'AHMED, SHAKIL ', 'AHMED, SYED A', 'AHMED, ZULFIQAR ', 'AHN, SAM S', 'AKBAR, MUHAMMAD S', 'AKHTAR, NAVEED ', 'ALAM, MUHAMMAD M', 'ALEXANDER, CHRISTOPHER ', 'ALI, AMJAD ', 'ALI, MAHMOOD ', 'ALI, MIR A', 'ALI, MIR M', 'ALI, MIR T', 'ALI, MOHAMED S', 'ALI, MOHAMMED ', 'ALI, MUHAMMAD ', 'ALI, SYED ', 'ALI,

In [21]:
# check for duplicate names
pacid_has_duplicate_names = len(doctors_multiple_pacid) != len(set(doctors_multiple_pacid))
print('Doctors with duplicate names:', pacid_has_duplicate_names)

Doctors with duplicate names: False


In [22]:
# check Enrollment ID, display only first 100
doctors_multiple_enrollmentid = doctors_enrollmentid[doctors_enrollmentid].index.tolist()
print('Doctors with multiple Enrollment IDs:', doctors_multiple_enrollmentid[:100])

Doctors with multiple Enrollment IDs: ['AARON, JOSHUA N', 'AARONSON, ORAN S', 'AARONSON, STEVEN P', 'AAS, PETER N', 'AASSAR, OLE SAMI', 'ABADI, CHRISTOPHER A', 'ABADIR, ADEL RAMSEY', 'ABALIHI, CAROL ', 'ABBAS, GHULAM ', 'ABBAS, JIHAD TAHSIN', 'ABBASI, ISRAR A', 'ABBOTT, DAVID J', 'ABBOUD, CAMILLE N', 'ABBOY, CHANDAR R', 'ABDALLA, ADEL ABAS MAHMOOD', 'ABDEL RAHMAN, AHMED B', 'ABDELHAK, TAMER ', 'ABDELMALEK, MARK ', 'ABDELMALEK, MOHEB ', 'ABDELMALIK, MAGED ', 'ABDU, ADIO ISHMAEL', 'ABDUL-MBACKE, MAKUNDA ', "ABDULLAH, MA'N A", 'ABDULLAH, MUHAMMAD ', 'ABDULSALAM, FARAH ZESHAN', 'ABEL, ARI DANIEL', 'ABELN, ELLEN L', 'ABEREGG, SCOTT KENNETH', 'ABERNATHY, ROBERT A', 'ABERNETHY, MICHAEL ', 'ABIODUN, OLUFEMI J', 'ABOODY, LINDA R', 'ABOU GHAZALA, TAREK ', 'ABOU-CHAKRA, IMAN SLEIMAN', 'ABOU-JAWDE, RONY M', 'ABOUCHALA, NABIL ', 'ABOUELENIN, KARIM ', 'ABOUESH, AHMED IBRAHI', 'ABOUFARES, ALI ', 'ABOUL MAGD, AHMED SAMEH', 'ABOUSSIE, MITCHELL ALBERT', 'ABOUSY, KHALID ', 'ABRAHAM, ANEY M', 'ABRAHAM, ED

In [23]:
# check for duplicate names
enrollmentid_has_duplicate_names = len(doctors_multiple_enrollmentid) != len(set(doctors_multiple_enrollmentid))
print('Doctors with duplicate names:', enrollmentid_has_duplicate_names)

Doctors with duplicate names: False


There are a lot of doctors that went to medical school listed as 'OTHER'

In [24]:
# check how many doctors went to a med school listed as 'OTHER'
doctors_df.loc[raw_df['Med_sch'] == 'OTHER']

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
74,1003002312,547203192,I20051018001123,HOPKINS,PATRICIA,T,NaN,F,MD,OTHER,...,NaN,WEYMOUTH,MA,2190,6177739198,Y,Y,MA021901843WE571XXSTXX300,"HOPKINS, PATRICIA T","571 MAIN ST, WEYMOUTH, MA 2190"
295,1003010349,1557388584,I20150822015059,SHAYA,WISSAM,,NaN,M,MD,OTHER,...,NaN,JACKSONVILLE,FL,32257,9043228555,Y,Y,FL322576013JA4235XRDXX300,"SHAYA, WISSAM","4235 SUNBEAM RD, JACKSONVILLE, FL 32257"
654,1003016353,5395777981,I20050902000649,SHARMA,CORDELIA,,NaN,F,MD,OTHER,...,NaN,BRONX,NY,10457,7189606214,Y,Y,NY104572545BR4422XAVEX300,"SHARMA, CORDELIA","4422 3RD AVE, BRONX, NY 10457"
1022,1003027335,8628020377,I20050210000834,KUSMIERZ,ZBIGNIEW,,NaN,M,MD,OTHER,...,NaN,HARLINGEN,TX,78550,9563651000,Y,Y,TX785503213HA5501XEXPR306,"KUSMIERZ, ZBIGNIEW","5501 S EXPRESSWAY SUITE 77, HARLINGEN, TX 78550"
1209,1003033689,7911006218,I20110707000382,TRIKHA,RUPAN,,NaN,M,MD,OTHER,...,NaN,LITTLE SILVER,NJ,7739,7325307730,Y,Y,NJ077391208LI39XXXAVEX300,"TRIKHA, RUPAN","39 SYCAMORE AVE, LITTLE SILVER, NJ 7739"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2479584,1992929947,2466345848,I20040209000148,CAMPBELL,ROBERT,L,NaN,M,MD,OTHER,...,NaN,FORT WORTH,TX,76115,8179247978,Y,Y,TX761151361FO546XXDRXX401,"CAMPBELL, ROBERT L","546 W SEMINARY DR SUITE A, FORT WORTH, TX 76115"
2481513,1992985360,9234189036,I20081028000661,BUJARSKI,KRZYSZTOF,A,NaN,M,MD,OTHER,...,NaN,LEBANON,NH,3756,6036505000,Y,Y,NH037561000LE1XXXXDRXX400,"BUJARSKI, KRZYSZTOF A","1 MEDICAL CTR DR, LEBANON, NH 3756"
2481514,1992985360,9234189036,I20081028000661,BUJARSKI,KRZYSZTOF,A,NaN,M,MD,OTHER,...,NaN,LEBANON,NH,3756,missing,Y,Y,NH037561000LE1XXXXDRXX400,"BUJARSKI, KRZYSZTOF A","1 MEDICAL CTR DR, LEBANON, NH 3756"
2481837,1992997332,5496776064,I20051209000751,ROTH,MARA,Y,NaN,F,MD,OTHER,...,NaN,SEATTLE,WA,98104,2067313000,Y,Y,WA981042420SE325XXAVEX300,"ROTH, MARA Y","325 9TH AVE, SEATTLE, WA 98104"


In [25]:
doctors_df['Med_sch'].value_counts()

OTHER                                                            132993
INDIANA UNIVERSITY SCHOOL OF MEDICINE                              5433
UNIVERSITY OF MINNESOTA MEDICAL SCHOOL                             5363
WAYNE STATE UNIVERSITY SCHOOL OF MEDICINE                          5140
UNIVERSITY OF ILLINOIS COLLEGE OF MED (CHI/PEOR/ROCK/CHM-URB)      5120
                                                                  ...  
KENT STATE UNIVERSITY COLLEGE OF PODIATRIC MEDICINE                   1
ENSWORTH MEDICAL COLLEGE                                              1
STARLING MEDICAL COLLEGE                                              1
PACIFIC UNIVERSITY - COLLEGE OF OPTOMETRY                             1
UNIVERSITY OF PASADENA COLLEGE OF CHIROPRACTIC                        1
Name: Med_sch, Length: 343, dtype: int64

It seems that 'OTHER' med school means that it was not located in the United States

In [26]:
# create a dataframe that does not have repeat doctors even though they may practice in multiple locations
# only take the first instance of the Full Name
unique_doctors_df = doctors_df.drop_duplicates(subset='full_name', keep='first')
unique_doctors_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
74,1003002312,547203192,I20051018001123,HOPKINS,PATRICIA,T,NaN,F,MD,OTHER,...,NaN,WEYMOUTH,MA,2190,6177739198,Y,Y,MA021901843WE571XXSTXX300,"HOPKINS, PATRICIA T","571 MAIN ST, WEYMOUTH, MA 2190"
111,1003003856,6305813148,I20040913000368,SANTIAGO,CYNTHIA,,NaN,F,MD,NEW YORK UNIVERSITY SCHOOL OF MEDICINE,...,NaN,PORT JEFFERSON STATION,NY,11776,6317326984,Y,M,NY117763336PO5XXXXAVEX300,"SANTIAGO, CYNTHIA","5 ROOSEVELT AVE, PORT JEFFERSON STATION, NY 11776"
119,1003004540,7416917000,I20041018000934,CABANELLAS,JENNINE,M,NaN,F,MD,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,...,NaN,BOYNTON BEACH,FL,33472,5617778946,M,M,FL334722900BO8198XRDXX403,"CABANELLAS, JENNINE M","8198 S JOG RD SUITE 102, BOYNTON BEACH, FL 33472"
174,1003006768,9032168018,I20220125002165,NEWMAN,MAUREEN,,NaN,F,MD,STATE UNIVERSITY OF NEW YORK AT BUFFALO SCHOOL...,...,Y,MERIDIAN,ID,83642,missing,Y,Y,ID836428531ME3025XLNXX400,"NEWMAN, MAUREEN","3025 W CHERRY LN, MERIDIAN, ID 83642"
295,1003010349,1557388584,I20150822015059,SHAYA,WISSAM,,NaN,M,MD,OTHER,...,NaN,JACKSONVILLE,FL,32257,9043228555,Y,Y,FL322576013JA4235XRDXX300,"SHAYA, WISSAM","4235 SUNBEAM RD, JACKSONVILLE, FL 32257"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2481438,1992981823,4284602434,I20040924000261,SKARBINSKI,JACK,,NaN,M,MD,STANFORD UNIVERSITY SCHOOL OF MEDICINE,...,NaN,OAKLAND,CA,94611,5107521000,Y,Y,CA946115312OA3801XSTXX300,"SKARBINSKI, JACK","3801 HOWE ST, OAKLAND, CA 94611"
2481513,1992985360,9234189036,I20081028000661,BUJARSKI,KRZYSZTOF,A,NaN,M,MD,OTHER,...,NaN,LEBANON,NH,3756,6036505000,Y,Y,NH037561000LE1XXXXDRXX400,"BUJARSKI, KRZYSZTOF A","1 MEDICAL CTR DR, LEBANON, NH 3756"
2481560,1992987358,1759346521,I20041201000614,FONDAL,MARIA,L,NaN,F,MD,WASHINGTON UNIVERSITY SCHOOL OF MEDICINE,...,NaN,ALBANY,GA,31701,2293121000,Y,Y,GA317011943AL417XXAVEX400,"FONDAL, MARIA L","417 W 3RD AVE, ALBANY, GA 31701"
2481782,1992995765,446246276,I20040426000309,APPEL,DEBRA,A,NaN,F,MD,BAYLOR COLLEGE OF MEDICINE,...,NaN,VALHALLA,NY,10595,missing,Y,Y,NY105951530VA100XXRDXX300,"APPEL, DEBRA A","100 WOODS RD, VALHALLA, NY 10595"


**There are 143,143 unique medical doctors and osteopathic doctors in this dataset.**

In [27]:
# check for duplicate FIRST and LAST names (ignoring middle initial)

# group by first and last names
npi_first_last_names = unique_doctors_df.groupby(['frst_nm', 'lst_nm'])

# filter groups with multiple unique NPIs
filtered_npi_first_last_names = npi_first_last_names.filter(lambda x: x['NPI'].nunique() > 1)

# Count the number of rows in the filtered groups
num_same_first_last_names = len(filtered_npi_first_last_names)

print('Doctors with same FIRST and LAST name:', num_same_first_last_names)

Doctors with same FIRST and LAST name: 20776


In [28]:
# sort alphabetically by last name
same_name_doctors_df = filtered_npi_first_last_names[filtered_npi_first_last_names.columns[:9]]
same_name_doctors_df.sort_values('lst_nm')

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred
1432225,1578722336,1658398508,I20151016001815,ABAD,JOHN,D,NaN,M,MD
875008,1356376800,7911904776,I20061109000027,ABAD,JOHN,P,NaN,M,MD
1397863,1568448850,6305940172,I20070404000243,ABBOTT,DAVID,A,NaN,M,MD
2310931,1932136173,7719880319,I20041013000719,ABBOTT,DAVID,J,NaN,M,MD
1422889,1578522991,5395707772,I20041028000385,ABDULKARIM,AHMAD,S,NaN,M,MD
...,...,...,...,...,...,...,...,...,...
1818885,1730183336,7719981430,I20060911000414,ZINK,ROBERT,S,NaN,M,MD
1431090,1578680070,5698792158,I20110303000560,ZISKO,JOHN,,NaN,M,MD
1296625,1528063740,9830003706,I20031114000464,ZISKO,JOHN,P,NaN,M,MD
1549577,1629182282,4880683853,I20040506001245,ZOLLINGER,CHARLES,J,NaN,M,MD


Check specialties for oncology since we are interested in tumor doctors.

In [29]:
specialty = 'ONCOLOGY'

In [30]:
oncology_df = unique_doctors_df[unique_doctors_df['pri_spec'].str.contains(specialty)]
oncology_df['pri_spec'].value_counts()

HEMATOLOGY/ONCOLOGY       2705
RADIATION ONCOLOGY        1484
MEDICAL ONCOLOGY          1240
GYNECOLOGICAL ONCOLOGY     379
SURGICAL ONCOLOGY          294
Name: pri_spec, dtype: int64

In [31]:
unique_doctors_df['sec_spec_1'].value_counts()

INTERNAL MEDICINE                                          14666
CRITICAL CARE (INTENSIVISTS)                                2825
CARDIOVASCULAR DISEASE (CARDIOLOGY)                         2568
PEDIATRIC MEDICINE                                          1913
EMERGENCY MEDICINE                                          1713
                                                           ...  
MEDICAL TOXICOLOGY                                            19
ADULT CONGENITAL HEART DISEASE (ACHD)                         17
PODIATRY                                                       7
HEMATOPOIETIC CELL TRANSPLANTATION AND CELLULAR THERAPY        6
CHIROPRACTIC                                                   3
Name: sec_spec_1, Length: 69, dtype: int64

In [32]:
unique_doctors_df['sec_spec_2'].value_counts()

INTERNAL MEDICINE                                          1954
PULMONARY DISEASE                                           420
SLEEP MEDICINE                                              269
PAIN MANAGEMENT                                             255
PEDIATRIC MEDICINE                                          178
                                                           ... 
MICROGRAPHIC DERMATOLOGIC SURGERY (MDS)                       1
OPTOMETRY                                                     1
CARDIAC ELECTROPHYSIOLOGY                                     1
HEMATOPOIETIC CELL TRANSPLANTATION AND CELLULAR THERAPY       1
ADVANCED HEART FAILURE AND TRANSPLANT CARDIOLOGY              1
Name: sec_spec_2, Length: 64, dtype: int64

In [33]:
unique_doctors_df['sec_spec_3'].value_counts()

SLEEP MEDICINE                          169
INTERNAL MEDICINE                        79
PAIN MANAGEMENT                          35
VASCULAR SURGERY                         34
PULMONARY DISEASE                        33
NUCLEAR MEDICINE                         33
PERIPHERAL VASCULAR DISEASE              33
OSTEOPATHIC MANIPULATIVE MEDICINE        18
PEDIATRIC MEDICINE                       18
MEDICAL ONCOLOGY                         18
SPORTS MEDICINE                          16
GERIATRIC MEDICINE                       16
PREVENTIVE MEDICINE                      15
THORACIC SURGERY                         15
INTERVENTIONAL PAIN MANAGEMENT           13
GENERAL PRACTICE                         12
PHYSICAL MEDICINE AND REHABILITATION     12
INTERVENTIONAL CARDIOLOGY                10
OBSTETRICS/GYNECOLOGY                     8
NEUROLOGY                                 8
INTERVENTIONAL RADIOLOGY                  8
HOSPICE/PALLIATIVE CARE                   7
SURGICAL ONCOLOGY               

In [34]:
unique_doctors_df['sec_spec_4'].value_counts()

VASCULAR SURGERY                        15
PERIPHERAL VASCULAR DISEASE             12
SLEEP MEDICINE                          10
PAIN MANAGEMENT                         10
PREVENTIVE MEDICINE                      7
PEDIATRIC MEDICINE                       6
INTERNAL MEDICINE                        6
GENERAL PRACTICE                         5
PULMONARY DISEASE                        5
GERIATRIC MEDICINE                       4
NUCLEAR MEDICINE                         4
INTERVENTIONAL PAIN MANAGEMENT           3
MEDICAL ONCOLOGY                         3
SPORTS MEDICINE                          2
OSTEOPATHIC MANIPULATIVE MEDICINE        2
GASTROENTEROLOGY                         2
SURGICAL ONCOLOGY                        2
NEUROLOGY                                2
PHYSICAL MEDICINE AND REHABILITATION     2
THORACIC SURGERY                         2
RHEUMATOLOGY                             2
OBSTETRICS/GYNECOLOGY                    2
MEDICAL GENETICS AND GENOMICS            1
FAMILY PRAC

In [35]:
unique_doctors_df['sec_spec_all'].value_counts()

INTERNAL MEDICINE                                       13906
CARDIOVASCULAR DISEASE (CARDIOLOGY)                      1921
PEDIATRIC MEDICINE                                       1892
CRITICAL CARE (INTENSIVISTS)                             1630
EMERGENCY MEDICINE                                       1506
                                                        ...  
GERIATRIC MEDICINE, HEMATOLOGY, MEDICAL ONCOLOGY            1
EMERGENCY MEDICINE, INTERVENTIONAL RADIOLOGY                1
HEMATOLOGY, INFECTIOUS DISEASE                              1
FAMILY PRACTICE, GERIATRIC MEDICINE, PAIN MANAGEMENT        1
ENDOCRINOLOGY, PEDIATRIC MEDICINE                           1
Name: sec_spec_all, Length: 915, dtype: int64

In [36]:
# filter for rows containing the DO and ONCOLOGY in the specified columns
do_oncology_df = unique_doctors_df[(unique_doctors_df['Cred'].str.contains('DO')) & (unique_doctors_df['pri_spec'].str.contains('ONCOLOGY'))]
do_oncology_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
18758,1003858234,5991704942,I20061220000415,ANDERSON,THOMAS,M,NaN,M,DO,"OHIO UNIVERSITY, COLLEGE OF OSTEOPATHIC MEDICINE",...,NaN,COLUMBUS,OH,43215,6145669489,Y,Y,OH432154701CO111XXAVEX400,"ANDERSON, THOMAS M","111 S GRANT AVE, COLUMBUS, OH 43215"
21311,1003892084,1557306073,I20220223001556,MISISCHIA,MARY,JANE,NaN,F,DO,OTHER,...,NaN,CHARLESTON,WV,25304,3043888380,Y,Y,WV253041334CH3415XSEXX400,"MISISCHIA, MARY JANE","3415 MACCORKLE AVE SE, CHARLESTON, WV 25304"
45507,1013932854,8123925393,I20031216000452,LUFKIN,ROBERT,C,NaN,M,DO,"UNIVERSITY OF NEW ENGLAND, COLLEGE OF OSTEO ME...",...,NaN,PORTLAND,OR,97213,5032397767,Y,Y,OR972132982PO5050XSTXX405,"LUFKIN, ROBERT C","5050 NE HOYT ST SUITE 256, PORTLAND, OR 97213"
46333,1013945708,8426061995,I20121019000438,WEISS,AARON,R,NaN,M,DO,PHILADELPHIA COLLEGE OF OSTEOPATHIC MEDICINE,...,NaN,PORTLAND,ME,4102,2076624618,Y,Y,ME041023134PO22XXXSTXX302,"WEISS, AARON R","22 BRAMHALL ST SUITE 2227, PORTLAND, ME 4102"
46965,1013955095,6002880994,I20220119002146,HALS,JESSICA,,NaN,F,DO,OTHER,...,NaN,JOHNSON CITY,NY,13790,6077636850,Y,Y,NY137903100JO30XXXSTXX301,"HALS, JESSICA","30 HARRISON ST SUITE 100, JOHNSON CITY, NY 13790"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337768,1942265517,5395710719,I20040830000641,BRUDIE,LORNA,A,NaN,F,DO,NEW YORK COLLEGE OF OSTEO MEDICINE OF NEW YORK...,...,NaN,KANSAS CITY,MO,64111,8169323300,Y,Y,MO641115965KA4321XSTXX302,"BRUDIE, LORNA A","4321 WASHINGTON ST SUITE 4000, KANSAS CITY, MO..."
2360850,1952325359,1254338767,I20180514000783,KIM,THEODORE,Y,NaN,M,DO,WEST VIRGINIA SCHOOL OF OSTEOPATHIC MEDICINE,...,NaN,ANCHORAGE,AK,99508,9075620321,Y,Y,AK995086904AN3851XSTXX303,"KIM, THEODORE Y","3851 PIPER ST SUITE U340, ANCHORAGE, AK 99508"
2444283,1982656971,1456362839,I20060523000141,LAYHE,ELIZABETH,M,NaN,F,DO,PHILADELPHIA COLLEGE OF OSTEOPATHIC MEDICINE,...,NaN,LANSING,MI,48910,5174834780,Y,Y,MI489102889LA405XXAVEX405,"LAYHE, ELIZABETH M",405 W GREENLAWN AVE 305 INGHAM REGIONAL MEDICA...
2446970,1982691689,3870487481,I20050202000638,SULLIVAN,JOSEPH,,NaN,M,DO,NOVA SOUTHEASTERN COLLEGE OF OSTEO MEDICINE,...,NaN,PALATKA,FL,32177,3863258140,Y,Y,FL321773811PA600XXDRXX300,"SULLIVAN, JOSEPH","600 ZEAGLER DR, PALATKA, FL 32177"


There are 112 unique DOs in the USA that PRIMARILY specialize in some form of oncology

In [37]:
# filter for rows containing the MD and ONCOLOGY in the specified columns
md_oncology_df = unique_doctors_df[(unique_doctors_df['Cred'].str.contains('MD')) & (unique_doctors_df['pri_spec'].str.contains('ONCOLOGY'))]
md_oncology_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
602,1003015207,5496725798,I20220418001335,SWANEY,ROBERT,,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,NaN,BILLINGS,MT,59101,4062382500,Y,Y,MT591010905BI801XXSTXX400,"SWANEY, ROBERT","801 N 29TH ST, BILLINGS, MT 59101"
1228,1003034760,7719954841,I20040916000688,GIRON,GLADYS,L,NaN,F,MD,"UNIVERSITY OF MIAMI, LM MILLER SCHOOL OF MEDICINE",...,NaN,MIAMI,FL,33176,7865967670,Y,Y,FL331762118MI8900XDRXX400,"GIRON, GLADYS L","8900 N KENDALL DR, MIAMI, FL 33176"
15040,1003800400,3971407982,I20031121000352,MOSBACHER,MARC,R,NaN,M,MD,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,...,NaN,CINCINNATI,OH,45236,5137512273,Y,Y,OH452362725CI4777XRDXX402,"MOSBACHER, MARC R","4777 E GALBRAITH RD SUITE 1, CINCINNATI, OH 45236"
15086,1003801283,7315965910,I20051102000667,JOHNSON,RUSSELL,FREDERICK,NaN,M,MD,UNIVERSITY OF ILLINOIS COLLEGE OF MED (CHI/PEO...,...,NaN,ELKHART,IN,46514,5745232733,Y,Y,IN465142450EL1215XAVEX303,"JOHNSON, RUSSELL FREDERICK","1215 LAWN AVE SUITE 120, ELKHART, IN 46514"
15120,1003801689,2163410457,I20040505000244,BHANJA,UTPAL,KUMAR,NaN,M,MD,OTHER,...,NaN,GALLIPOLIS,OH,45631,7404465474,Y,Y,OH456311539GA170XXPIKE300,"BHANJA, UTPAL KUMAR","170 JACKSON PIKE, GALLIPOLIS, OH 45631"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477684,1992876452,4284603218,I20040927001218,GILBERT,MICHELLE,HEATHER,NaN,F,MD,UNIVERSITY OF UTAH SCHOOL OF MEDICINE,...,NaN,FILLMORE,UT,84631,4357435555,Y,Y,UT846315134FI700XX99XX400,"GILBERT, MICHELLE HEATHER","700 S HWY 99, FILLMORE, UT 84631"
2477831,1992881403,2860479649,I20040630000327,KWONG,MYRON,S,NaN,M,MD,NEW YORK MEDICAL COLLEGE,...,NaN,SANTA CLARA,CA,95051,4088511000,Y,Y,CA950515173SA700XXEXPY300,"KWONG, MYRON S","700 LAWRENCE EXPY, SANTA CLARA, CA 95051"
2478204,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,NaN,NEW YORK,NY,10021,6469622012,Y,Y,NY100219800NE520XXSTXX400,"LEONARD, JOHN P","520 E 70TH ST, NEW YORK, NY 10021"
2478320,1992895510,9931113818,I20060131000088,SHAMI,PAUL,J,NaN,M,MD,OTHER,...,NaN,SALT LAKE CITY,UT,84112,8015818793,Y,Y,UT841125500SA1950XDRXX500,"SHAMI, PAUL J","1950 CIRCLE OF HOPE DR, SALT LAKE CITY, UT 84112"


There are 4,318 unique MDs in the USA that PRIMARILY specialize in some form of oncology

In [38]:
# filter for rows containing ONCOLOGY in the specified columns (this df contains both MDs and DOs)
oncology_df = unique_doctors_df[(unique_doctors_df['pri_spec'].str.contains('ONCOLOGY')) | 
                                (unique_doctors_df['sec_spec_1'].str.contains('ONCOLOGY')) |
                                (unique_doctors_df['sec_spec_2'].str.contains('ONCOLOGY')) |
                                (unique_doctors_df['sec_spec_3'].str.contains('ONCOLOGY')) |
                                (unique_doctors_df['sec_spec_4'].str.contains('ONCOLOGY')) |
                                (unique_doctors_df['sec_spec_all'].str.contains('ONCOLOGY'))]
oncology_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
602,1003015207,5496725798,I20220418001335,SWANEY,ROBERT,,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,NaN,BILLINGS,MT,59101,4062382500,Y,Y,MT591010905BI801XXSTXX400,"SWANEY, ROBERT","801 N 29TH ST, BILLINGS, MT 59101"
1228,1003034760,7719954841,I20040916000688,GIRON,GLADYS,L,NaN,F,MD,"UNIVERSITY OF MIAMI, LM MILLER SCHOOL OF MEDICINE",...,NaN,MIAMI,FL,33176,7865967670,Y,Y,FL331762118MI8900XDRXX400,"GIRON, GLADYS L","8900 N KENDALL DR, MIAMI, FL 33176"
15040,1003800400,3971407982,I20031121000352,MOSBACHER,MARC,R,NaN,M,MD,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,...,NaN,CINCINNATI,OH,45236,5137512273,Y,Y,OH452362725CI4777XRDXX402,"MOSBACHER, MARC R","4777 E GALBRAITH RD SUITE 1, CINCINNATI, OH 45236"
15086,1003801283,7315965910,I20051102000667,JOHNSON,RUSSELL,FREDERICK,NaN,M,MD,UNIVERSITY OF ILLINOIS COLLEGE OF MED (CHI/PEO...,...,NaN,ELKHART,IN,46514,5745232733,Y,Y,IN465142450EL1215XAVEX303,"JOHNSON, RUSSELL FREDERICK","1215 LAWN AVE SUITE 120, ELKHART, IN 46514"
15120,1003801689,2163410457,I20040505000244,BHANJA,UTPAL,KUMAR,NaN,M,MD,OTHER,...,NaN,GALLIPOLIS,OH,45631,7404465474,Y,Y,OH456311539GA170XXPIKE300,"BHANJA, UTPAL KUMAR","170 JACKSON PIKE, GALLIPOLIS, OH 45631"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2477684,1992876452,4284603218,I20040927001218,GILBERT,MICHELLE,HEATHER,NaN,F,MD,UNIVERSITY OF UTAH SCHOOL OF MEDICINE,...,NaN,FILLMORE,UT,84631,4357435555,Y,Y,UT846315134FI700XX99XX400,"GILBERT, MICHELLE HEATHER","700 S HWY 99, FILLMORE, UT 84631"
2477831,1992881403,2860479649,I20040630000327,KWONG,MYRON,S,NaN,M,MD,NEW YORK MEDICAL COLLEGE,...,NaN,SANTA CLARA,CA,95051,4088511000,Y,Y,CA950515173SA700XXEXPY300,"KWONG, MYRON S","700 LAWRENCE EXPY, SANTA CLARA, CA 95051"
2478204,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,NaN,NEW YORK,NY,10021,6469622012,Y,Y,NY100219800NE520XXSTXX400,"LEONARD, JOHN P","520 E 70TH ST, NEW YORK, NY 10021"
2478320,1992895510,9931113818,I20060131000088,SHAMI,PAUL,J,NaN,M,MD,OTHER,...,NaN,SALT LAKE CITY,UT,84112,8015818793,Y,Y,UT841125500SA1950XDRXX500,"SHAMI, PAUL J","1950 CIRCLE OF HOPE DR, SALT LAKE CITY, UT 84112"


In [39]:
# check zip codes
oncology_df['zip'].value_counts()

77030    93
2114     56
44195    50
2215     45
2115     42
         ..
92128     1
38019     1
48307     1
959       1
11530     1
Name: zip, Length: 2330, dtype: int64

In [40]:
# filter for rows that do NOT contain ONCOLOGY in as a primary specialty
secondary_oncology_df = oncology_df[~(oncology_df['pri_spec'].str.contains('ONCOLOGY'))]
secondary_oncology_df['pri_spec']

19366                                        GENERAL SURGERY
19421                                        GENERAL SURGERY
20607      HEMATOPOIETIC CELL TRANSPLANTATION AND CELLULA...
22172                                      INTERNAL MEDICINE
22276                                        GENERAL SURGERY
                                 ...                        
2454243                                    INTERNAL MEDICINE
2473017                                      GENERAL SURGERY
2474313                                            NEUROLOGY
2475012                                   INFECTIOUS DISEASE
2476962                                      GENERAL SURGERY
Name: pri_spec, Length: 672, dtype: object

There are 478 MDs or DOs that specialize in oncology but not as their primary specialty

In [41]:
oncology_df['st'].value_counts()

CA    668
TX    536
NY    532
FL    469
PA    341
IL    311
MA    302
OH    276
NJ    226
NC    206
MI    200
GA    168
WA    164
VA    142
WI    142
AZ    141
MO    130
MN    129
IN    118
CO    113
MD    111
TN    109
OR     87
KY     84
AL     82
CT     80
SC     77
LA     75
AR     52
UT     51
OK     51
NE     49
MS     45
IA     45
NH     42
KS     41
NV     40
NM     35
RI     33
PR     32
ME     30
HI     28
WV     26
MT     25
DC     22
ID     18
VT     16
ND     16
AK     16
DE     15
SD     12
WY      9
GU      4
VI      2
Name: st, dtype: int64

In [42]:
# export cleaned UNIQUE oncologist data
oncology_df.to_csv('/kaggle/working/unique_oncologists.csv', index=False)

we have one file with all of the unique oncologists but we also want to have all of their practice locations

In [43]:
# create a dataframe that DOES have repeat doctors because we need all of their practice locations
# filter for rows containing ONCOLOGY in the specified columns (this df contains both MDs and DOs)
all_oncologists_df = doctors_df[(doctors_df['pri_spec'].str.contains('ONCOLOGY')) | 
                                (doctors_df['sec_spec_1'].str.contains('ONCOLOGY')) |
                                (doctors_df['sec_spec_2'].str.contains('ONCOLOGY')) |
                                (doctors_df['sec_spec_3'].str.contains('ONCOLOGY')) |
                                (doctors_df['sec_spec_4'].str.contains('ONCOLOGY')) |
                                (doctors_df['sec_spec_all'].str.contains('ONCOLOGY'))]

all_oncologists_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ln_2_sprs,cty,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address
602,1003015207,5496725798,I20220418001335,SWANEY,ROBERT,,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,NaN,BILLINGS,MT,59101,4062382500,Y,Y,MT591010905BI801XXSTXX400,"SWANEY, ROBERT","801 N 29TH ST, BILLINGS, MT 59101"
603,1003015207,5496725798,I20230126001462,SWANEY,ROBERT,,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,NaN,NEWBURGH,IN,47630,8124711200,Y,Y,IN476308909NE3699XRDXX300,"SWANEY, ROBERT","3699 EPWORTH RD, NEWBURGH, IN 47630"
1228,1003034760,7719954841,I20040916000688,GIRON,GLADYS,L,NaN,F,MD,"UNIVERSITY OF MIAMI, LM MILLER SCHOOL OF MEDICINE",...,NaN,MIAMI,FL,33176,7865967670,Y,Y,FL331762118MI8900XDRXX400,"GIRON, GLADYS L","8900 N KENDALL DR, MIAMI, FL 33176"
15040,1003800400,3971407982,I20031121000352,MOSBACHER,MARC,R,NaN,M,MD,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,...,NaN,CINCINNATI,OH,45236,5137512273,Y,Y,OH452362725CI4777XRDXX402,"MOSBACHER, MARC R","4777 E GALBRAITH RD SUITE 1, CINCINNATI, OH 45236"
15086,1003801283,7315965910,I20051102000667,JOHNSON,RUSSELL,FREDERICK,NaN,M,MD,UNIVERSITY OF ILLINOIS COLLEGE OF MED (CHI/PEO...,...,NaN,ELKHART,IN,46514,5745232733,Y,Y,IN465142450EL1215XAVEX303,"JOHNSON, RUSSELL FREDERICK","1215 LAWN AVE SUITE 120, ELKHART, IN 46514"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2478205,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,NaN,NEW YORK,NY,10021,2127466575,Y,Y,NY100219800NE520XXSTXX409,"LEONARD, JOHN P","520 E 70TH ST SUITE 651, NEW YORK, NY 10021"
2478206,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,NaN,NEW YORK,NY,10065,missing,Y,Y,NY100654870NE525XXSTXX400,"LEONARD, JOHN P","525 E 68TH ST, NEW YORK, NY 10065"
2478207,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,NaN,NEW YORK,NY,10065,2127463558,Y,Y,NY100654870NE525XXSTXX406,"LEONARD, JOHN P","525 E 68TH ST M6, NEW YORK, NY 10065"
2478320,1992895510,9931113818,I20060131000088,SHAMI,PAUL,J,NaN,M,MD,OTHER,...,NaN,SALT LAKE CITY,UT,84112,8015818793,Y,Y,UT841125500SA1950XDRXX500,"SHAMI, PAUL J","1950 CIRCLE OF HOPE DR, SALT LAKE CITY, UT 84112"


In [45]:
# exceeded rate-limit for geocoder API so import previously generated file
all_oncologists_df = pd.read_csv('/kaggle/input/all-oncologists-lat-long/all_oncologists_lat_long.csv')
all_oncologists_df

/tmp/ipykernel_32/2758277784.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  all_oncologists_df = pd.read_csv('/kaggle/input/all-oncologists-lat-long/all_oncologists_lat_long.csv')


,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,st,zip,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address,latitude,longitude
0,1003015207,5496725798,I20220418001335,SWANEY,ROBERT,NaN,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,MT,59101,4062382500,Y,Y,MT591010905BI801XXSTXX400,"SWANEY, ROBERT","801 N 29TH ST, BILLINGS, MT 59101",45.787789,-108.512586
1,1003015207,5496725798,I20230126001462,SWANEY,ROBERT,NaN,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,IN,47630,8124711200,Y,Y,IN476308909NE3699XRDXX300,"SWANEY, ROBERT","3699 EPWORTH RD, NEWBURGH, IN 47630",37.981127,-87.442704
2,1003034760,7719954841,I20040916000688,GIRON,GLADYS,L,NaN,F,MD,"UNIVERSITY OF MIAMI, LM MILLER SCHOOL OF MEDICINE",...,FL,33176,7865967670,Y,Y,FL331762118MI8900XDRXX400,"GIRON, GLADYS L","8900 N KENDALL DR, MIAMI, FL 33176",25.683594,-80.338203
3,1003800400,3971407982,I20031121000352,MOSBACHER,MARC,R,NaN,M,MD,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,...,OH,45236,5137512273,Y,Y,OH452362725CI4777XRDXX402,"MOSBACHER, MARC R","4777 E GALBRAITH RD SUITE 1, CINCINNATI, OH 45236",39.205869,-84.381013
4,1003801283,7315965910,I20051102000667,JOHNSON,RUSSELL,FREDERICK,NaN,M,MD,UNIVERSITY OF ILLINOIS COLLEGE OF MED (CHI/PEO...,...,IN,46514,5745232733,Y,Y,IN465142450EL1215XAVEX303,"JOHNSON, RUSSELL FREDERICK","1215 LAWN AVE SUITE 120, ELKHART, IN 46514",41.679288,-85.993260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16517,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,NY,10021,2127466575,Y,Y,NY100219800NE520XXSTXX409,"LEONARD, JOHN P","520 E 70TH ST SUITE 651, NEW YORK, NY 10021",40.765090,-73.953755
16518,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,NY,10065,missing,Y,Y,NY100654870NE525XXSTXX400,"LEONARD, JOHN P","525 E 68TH ST, NEW YORK, NY 10065",40.764777,-73.954801
16519,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,NY,10065,2127463558,Y,Y,NY100654870NE525XXSTXX406,"LEONARD, JOHN P","525 E 68TH ST M6, NEW YORK, NY 10065",40.764301,-73.954280
16520,1992895510,9931113818,I20060131000088,SHAMI,PAUL,J,NaN,M,MD,OTHER,...,UT,84112,8015818793,Y,Y,UT841125500SA1950XDRXX500,"SHAMI, PAUL J","1950 CIRCLE OF HOPE DR, SALT LAKE CITY, UT 84112",40.773364,-111.834893


### US Zip Codes

In [46]:
raw_zip_df = gpd.read_file('/kaggle/input/us-zip-code-boundaries/tl_2019_us_zcta510.shp')
raw_zip_df

,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,43451,43451,B5,G6350,S,63484186,157689,+41.3183010,-083.6174935,"POLYGON ((-83.70873 41.32733, -83.70815 41.327..."
1,43452,43452,B5,G6350,S,121522304,13721730,+41.5157923,-082.9809454,"POLYGON ((-83.08698 41.53780, -83.08256 41.537..."
2,43456,43456,B5,G6350,S,9320975,1003775,+41.6318300,-082.8393923,"MULTIPOLYGON (((-82.83558 41.71082, -82.83515 ..."
3,43457,43457,B5,G6350,S,48004681,0,+41.2673301,-083.4274872,"POLYGON ((-83.49650 41.25371, -83.48382 41.253..."
4,43458,43458,B5,G6350,S,2573816,39915,+41.5304461,-083.2133648,"POLYGON ((-83.22229 41.53102, -83.22228 41.532..."
...,...,...,...,...,...,...,...,...,...,...
33139,84044,84044,B5,G6350,S,123283611,69912363,+40.7201095,-112.1549645,"POLYGON ((-112.26022 40.76909, -112.25333 40.7..."
33140,84045,84045,B5,G6350,S,52780589,105248,+40.3364908,-111.9111905,"MULTIPOLYGON (((-111.92421 40.17034, -111.9240..."
33141,84046,84046,B5,G6350,S,333205012,4460752,+40.9207693,-109.8310453,"POLYGON ((-110.00072 40.99745, -110.00036 40.9..."
33142,84047,84047,B5,G6350,S,16172461,0,+40.6152301,-111.8907499,"POLYGON ((-111.92141 40.62772, -111.92134 40.6..."


In [47]:
# rename columns
zip_codes_df = raw_zip_df.loc[:, ['ZCTA5CE10','INTPTLAT10','INTPTLON10','geometry']]
zip_codes_df = zip_codes_df.rename(columns={'ZCTA5CE10':'zip',
                                            'INTPTLAT10':'centroid_latitude',
                                            'INTPTLON10':'centroid_longitude'})
zip_codes_df

,zip,centroid_latitude,centroid_longitude,geometry
0,43451,+41.3183010,-083.6174935,"POLYGON ((-83.70873 41.32733, -83.70815 41.327..."
1,43452,+41.5157923,-082.9809454,"POLYGON ((-83.08698 41.53780, -83.08256 41.537..."
2,43456,+41.6318300,-082.8393923,"MULTIPOLYGON (((-82.83558 41.71082, -82.83515 ..."
3,43457,+41.2673301,-083.4274872,"POLYGON ((-83.49650 41.25371, -83.48382 41.253..."
4,43458,+41.5304461,-083.2133648,"POLYGON ((-83.22229 41.53102, -83.22228 41.532..."
...,...,...,...,...
33139,84044,+40.7201095,-112.1549645,"POLYGON ((-112.26022 40.76909, -112.25333 40.7..."
33140,84045,+40.3364908,-111.9111905,"MULTIPOLYGON (((-111.92421 40.17034, -111.9240..."
33141,84046,+40.9207693,-109.8310453,"POLYGON ((-110.00072 40.99745, -110.00036 40.9..."
33142,84047,+40.6152301,-111.8907499,"POLYGON ((-111.92141 40.62772, -111.92134 40.6..."


In [48]:
zip_codes_df.dtypes

zip                     object
centroid_latitude       object
centroid_longitude      object
geometry              geometry
dtype: object

In [49]:
# convert columns to proper dtypes
zip_codes_df['zip'] = zip_codes_df['zip'].astype(int)
zip_codes_df['centroid_latitude'] = zip_codes_df['centroid_latitude'].astype(float)
zip_codes_df['centroid_longitude'] = zip_codes_df['centroid_longitude'].astype(float)

In [50]:
# combine based on zip_code and add geometry and centroid
oncologists_df = pd.merge(all_oncologists_df, zip_codes_df[['zip', 'centroid_latitude', 'centroid_longitude']], on='zip', how='left')
oncologists_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,phn_numbr,ind_assgn,grp_assgn,adrs_id,full_name,full_address,latitude,longitude,centroid_latitude,centroid_longitude
0,1003015207,5496725798,I20220418001335,SWANEY,ROBERT,NaN,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,4062382500,Y,Y,MT591010905BI801XXSTXX400,"SWANEY, ROBERT","801 N 29TH ST, BILLINGS, MT 59101",45.787789,-108.512586,45.613054,-108.390305
1,1003015207,5496725798,I20230126001462,SWANEY,ROBERT,NaN,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,8124711200,Y,Y,IN476308909NE3699XRDXX300,"SWANEY, ROBERT","3699 EPWORTH RD, NEWBURGH, IN 47630",37.981127,-87.442704,37.952897,-87.346470
2,1003034760,7719954841,I20040916000688,GIRON,GLADYS,L,NaN,F,MD,"UNIVERSITY OF MIAMI, LM MILLER SCHOOL OF MEDICINE",...,7865967670,Y,Y,FL331762118MI8900XDRXX400,"GIRON, GLADYS L","8900 N KENDALL DR, MIAMI, FL 33176",25.683594,-80.338203,25.659878,-80.358354
3,1003800400,3971407982,I20031121000352,MOSBACHER,MARC,R,NaN,M,MD,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,...,5137512273,Y,Y,OH452362725CI4777XRDXX402,"MOSBACHER, MARC R","4777 E GALBRAITH RD SUITE 1, CINCINNATI, OH 45236",39.205869,-84.381013,39.209810,-84.397302
4,1003801283,7315965910,I20051102000667,JOHNSON,RUSSELL,FREDERICK,NaN,M,MD,UNIVERSITY OF ILLINOIS COLLEGE OF MED (CHI/PEO...,...,5745232733,Y,Y,IN465142450EL1215XAVEX303,"JOHNSON, RUSSELL FREDERICK","1215 LAWN AVE SUITE 120, ELKHART, IN 46514",41.679288,-85.993260,41.722703,-85.977009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16517,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,2127466575,Y,Y,NY100219800NE520XXSTXX409,"LEONARD, JOHN P","520 E 70TH ST SUITE 651, NEW YORK, NY 10021",40.765090,-73.953755,40.769224,-73.958741
16518,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,missing,Y,Y,NY100654870NE525XXSTXX400,"LEONARD, JOHN P","525 E 68TH ST, NEW YORK, NY 10065",40.764777,-73.954801,40.764628,-73.963144
16519,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,2127463558,Y,Y,NY100654870NE525XXSTXX406,"LEONARD, JOHN P","525 E 68TH ST M6, NEW YORK, NY 10065",40.764301,-73.954280,40.764628,-73.963144
16520,1992895510,9931113818,I20060131000088,SHAMI,PAUL,J,NaN,M,MD,OTHER,...,8015818793,Y,Y,UT841125500SA1950XDRXX500,"SHAMI, PAUL J","1950 CIRCLE OF HOPE DR, SALT LAKE CITY, UT 84112",40.773364,-111.834893,40.765076,-111.841478


In [51]:
oncologists_df.columns

Index(['NPI', 'Ind_PAC_ID', 'Ind_enrl_ID', 'lst_nm', 'frst_nm', 'mid_nm',
       'suff', 'gndr', 'Cred', 'Med_sch', 'Grd_yr', 'pri_spec', 'sec_spec_1',
       'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all', 'Telehlth',
       'org_nm', 'org_pac_id', 'num_org_mem', 'adr_ln_1', 'adr_ln_2',
       'ln_2_sprs', 'cty', 'st', 'zip', 'phn_numbr', 'ind_assgn', 'grp_assgn',
       'adrs_id', 'full_name', 'full_address', 'latitude', 'longitude',
       'centroid_latitude', 'centroid_longitude'],
      dtype='object')

In [52]:
# calculate the distance from zip code centroid to address latitude and longitude
def calculate_distance(point1, point2):
    return cdist([point1], [point2])[0][0]

In [53]:
# create a network graph using NetworkX
graph = nx.Graph()

In [54]:
# iterate over the addresses in the DataFrame
for index, row in oncologists_df.iterrows():
    address_lat = row['latitude']  # Latitude of the address
    address_lon = row['longitude']  # Longitude of the address

    # get the zip code for the address
    zip_code = row['zip']

    # get the centroid of the zip code from the zip_code_centroids dataframe
    zip_code_centroid_lat = row['centroid_latitude']
    zip_code_centroid_long = row['centroid_longitude']
    zip_code_centroid = (zip_code_centroid_lat, zip_code_centroid_long)

    # calculate the distance between the address and the zip code centroid
    distance = calculate_distance((address_lat, address_lon), (zip_code_centroid_lat, zip_code_centroid_long))
    
    # add the new value to the new column
    oncologists_df.loc[index, 'distance_from_center_zip'] = distance
    
#     print(f'{zip_code} - {zip_code_centroid} - address: ({address_lat}, {address_lon}) - distance: {distance}')
    
    # Add the zip code centroid as a node in the graph
    graph.add_node(zip_code_centroid)

    # Add an edge between the address and the zip code centroid with the distance as the weight
    graph.add_edge((address_lat, address_lon), zip_code_centroid, weight=distance)

In [55]:
oncologists_df

,NPI,Ind_PAC_ID,Ind_enrl_ID,lst_nm,frst_nm,mid_nm,suff,gndr,Cred,Med_sch,...,ind_assgn,grp_assgn,adrs_id,full_name,full_address,latitude,longitude,centroid_latitude,centroid_longitude,distance_from_center_zip
0,1003015207,5496725798,I20220418001335,SWANEY,ROBERT,NaN,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,Y,Y,MT591010905BI801XXSTXX400,"SWANEY, ROBERT","801 N 29TH ST, BILLINGS, MT 59101",45.787789,-108.512586,45.613054,-108.390305,0.213273
1,1003015207,5496725798,I20230126001462,SWANEY,ROBERT,NaN,NaN,M,MD,UNIVERSITY OF MICHIGAN MEDICAL SCHOOL,...,Y,Y,IN476308909NE3699XRDXX300,"SWANEY, ROBERT","3699 EPWORTH RD, NEWBURGH, IN 47630",37.981127,-87.442704,37.952897,-87.346470,0.100289
2,1003034760,7719954841,I20040916000688,GIRON,GLADYS,L,NaN,F,MD,"UNIVERSITY OF MIAMI, LM MILLER SCHOOL OF MEDICINE",...,Y,Y,FL331762118MI8900XDRXX400,"GIRON, GLADYS L","8900 N KENDALL DR, MIAMI, FL 33176",25.683594,-80.338203,25.659878,-80.358354,0.031121
3,1003800400,3971407982,I20031121000352,MOSBACHER,MARC,R,NaN,M,MD,COLUMBIA UNIVERSITY COLLEGE OF PHYSICIANS AND ...,...,Y,Y,OH452362725CI4777XRDXX402,"MOSBACHER, MARC R","4777 E GALBRAITH RD SUITE 1, CINCINNATI, OH 45236",39.205869,-84.381013,39.209810,-84.397302,0.016759
4,1003801283,7315965910,I20051102000667,JOHNSON,RUSSELL,FREDERICK,NaN,M,MD,UNIVERSITY OF ILLINOIS COLLEGE OF MED (CHI/PEO...,...,Y,Y,IN465142450EL1215XAVEX303,"JOHNSON, RUSSELL FREDERICK","1215 LAWN AVE SUITE 120, ELKHART, IN 46514",41.679288,-85.993260,41.722703,-85.977009,0.046357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16517,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,Y,Y,NY100219800NE520XXSTXX409,"LEONARD, JOHN P","520 E 70TH ST SUITE 651, NEW YORK, NY 10021",40.765090,-73.953755,40.769224,-73.958741,0.006477
16518,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,Y,Y,NY100654870NE525XXSTXX400,"LEONARD, JOHN P","525 E 68TH ST, NEW YORK, NY 10065",40.764777,-73.954801,40.764628,-73.963144,0.008344
16519,1992892079,4486660370,I20060301000795,LEONARD,JOHN,P,NaN,M,MD,UNIVERSITY OF VIRGINIA SCHOOL OF MEDICINE,...,Y,Y,NY100654870NE525XXSTXX406,"LEONARD, JOHN P","525 E 68TH ST M6, NEW YORK, NY 10065",40.764301,-73.954280,40.764628,-73.963144,0.008870
16520,1992895510,9931113818,I20060131000088,SHAMI,PAUL,J,NaN,M,MD,OTHER,...,Y,Y,UT841125500SA1950XDRXX500,"SHAMI, PAUL J","1950 CIRCLE OF HOPE DR, SALT LAKE CITY, UT 84112",40.773364,-111.834893,40.765076,-111.841478,0.010585


In [56]:
oncologists_df.columns

Index(['NPI', 'Ind_PAC_ID', 'Ind_enrl_ID', 'lst_nm', 'frst_nm', 'mid_nm',
       'suff', 'gndr', 'Cred', 'Med_sch', 'Grd_yr', 'pri_spec', 'sec_spec_1',
       'sec_spec_2', 'sec_spec_3', 'sec_spec_4', 'sec_spec_all', 'Telehlth',
       'org_nm', 'org_pac_id', 'num_org_mem', 'adr_ln_1', 'adr_ln_2',
       'ln_2_sprs', 'cty', 'st', 'zip', 'phn_numbr', 'ind_assgn', 'grp_assgn',
       'adrs_id', 'full_name', 'full_address', 'latitude', 'longitude',
       'centroid_latitude', 'centroid_longitude', 'distance_from_center_zip'],
      dtype='object')

In [57]:
oncologists_df['distance_from_center_zip'].value_counts()

0.020850    248
0.004782    186
0.000697    134
0.035263    130
0.009924     98
           ... 
0.170113      1
0.021645      1
0.015963      1
0.024983      1
0.029720      1
Name: distance_from_center_zip, Length: 6068, dtype: int64

In [58]:
# export cleaned UNIQUE oncologist data
oncologists_df.to_csv('/kaggle/working/all_oncologists.csv', index=False)

## Generate random addresses for training data

In [59]:
pip install random-address

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 2.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [60]:
import random_address
from random_address import real_random_address

In [61]:
# Generate a dictionary with valid random address information
real_random_address()

{'address1': '6730 North 59th Drive',
 'address2': '#4',
 'city': 'Glendale',
 'state': 'AZ',
 'postalCode': '85301',
 'coordinates': {'lat': 33.5360084, 'lng': -112.1882161}}

In [62]:
# create a list to hold all random addresses
random_address_list = []
for i in range(500):
    random_address_list.append(real_random_address())
    
print(len(random_address_list))

# append to a dataframe
random_address_df = pd.DataFrame()

# Create a DataFrame using pd.concat()
random_address_df = pd.concat([pd.DataFrame(address, index=[0]) for address in random_address_list], ignore_index=True)

# Print the DataFrame
random_address_df

500


,address1,address2,city,state,postalCode,coordinates
0,9903 West 53rd Place,,Arvada,CO,80002,NaN
1,7770 West Bell Road,,Glendale,AZ,85308,NaN
2,922 Urciolo Court,,Annapolis,MD,21401,NaN
3,115 Saint John Street,,Manchester,CT,06040,NaN
4,3600 San Sebastian Court,,Punta Gorda,FL,33950,NaN
...,...,...,...,...,...,...
495,5611 North 61st Avenue,,Glendale,AZ,85301,NaN
496,711 Tashanna Lane,,Southport,FL,32409,NaN
497,1803 Hickory Avenue,UNIT C,Panama City,FL,32405,NaN
498,334 North Cove Boulevard,,Panama City,FL,32401,NaN


In [63]:
# formatting

# fill the address2 column with blanks instead of NaNs
random_address_df['address2'] = random_address_df['address2'].fillna('')

# drop coordinates column
random_address_df = random_address_df.drop('coordinates', axis=1)

# rename postal code to zip
random_address_df = random_address_df.rename(columns={'postalCode': 'zip',})

# create full address column
random_address_df['full_address'] = random_address_df['address1'] + ' ' + random_address_df['address2'] + ', ' + random_address_df['city'] + ', ' + random_address_df['state'] + ' ' + random_address_df['zip']
                
random_address_df

,address1,address2,city,state,zip,full_address
0,9903 West 53rd Place,,Arvada,CO,80002,"9903 West 53rd Place , Arvada, CO 80002"
1,7770 West Bell Road,,Glendale,AZ,85308,"7770 West Bell Road , Glendale, AZ 85308"
2,922 Urciolo Court,,Annapolis,MD,21401,"922 Urciolo Court , Annapolis, MD 21401"
3,115 Saint John Street,,Manchester,CT,06040,"115 Saint John Street , Manchester, CT 06040"
4,3600 San Sebastian Court,,Punta Gorda,FL,33950,"3600 San Sebastian Court , Punta Gorda, FL 33950"
...,...,...,...,...,...,...
495,5611 North 61st Avenue,,Glendale,AZ,85301,"5611 North 61st Avenue , Glendale, AZ 85301"
496,711 Tashanna Lane,,Southport,FL,32409,"711 Tashanna Lane , Southport, FL 32409"
497,1803 Hickory Avenue,UNIT C,Panama City,FL,32405,"1803 Hickory Avenue UNIT C, Panama City, FL 32405"
498,334 North Cove Boulevard,,Panama City,FL,32401,"334 North Cove Boulevard , Panama City, FL 32401"


In [64]:
# export randomly generated patient training data
random_address_df.to_csv('/kaggle/working/training.csv', index=False)